In [71]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

In [72]:
# From rotation experiment
def rotate_about_center(src, angle, scale=1.):
    w = src.shape[1]
    h = src.shape[0]
    rangle = np.deg2rad(angle)  # angle in radians
    # now calculate new image width and height
    nw = (abs(np.sin(rangle)*h) + abs(np.cos(rangle)*w))*scale
    nh = (abs(np.cos(rangle)*h) + abs(np.sin(rangle)*w))*scale
    # ask OpenCV for the rotation matrix
    rot_mat = cv2.getRotationMatrix2D((nw*0.5, nh*0.5), angle, scale)
    # calculate the move from the old center to the new center combined
    # with the rotation
    rot_move = np.dot(rot_mat, np.array([(nw-w)*0.5, (nh-h)*0.5,0]))
    # the move only affects the translation, so update the translation
    # part of the transform
    rot_mat[0,2] += rot_move[0]
    rot_mat[1,2] += rot_move[1]
    return cv2.warpAffine(src, rot_mat, (int(math.ceil(nw)), int(math.ceil(nh))), flags=cv2.INTER_LANCZOS4)

In [89]:
img = cv2.imread("136.jpg", 0)

In [90]:
ret,thresh1 = cv2.threshold(img,200,255,cv2.THRESH_BINARY)


plt.imshow(thresh1)
plt.show()
#cv2.imshow('img',dst)

In [91]:
# Nem tuls fasza...
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
plt.imshow(th2)
plt.show()

In [92]:
# This is the winner
ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
plt.imshow(th2)
plt.show()

In [93]:
contours, hierarchy = cv2.findContours(th2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [94]:
cnt = contours[0]
for act_cnt in contours:
    if cv2.contourArea(act_cnt) > cv2.contourArea(cnt):
        cnt = act_cnt

In [95]:
rect = cv2.minAreaRect(cnt)
box = cv2.cv.BoxPoints(rect)
box = np.int0(box)
#cv2.drawContours(img,[box],0,(255),0)

In [96]:
cv2.imwrite("result.jpg", img)


True

In [97]:
rect

((228.40533447265625, 65.18756103515625),
 (448.5104064941406, 53.477638244628906),
 -3.4682295322418213)

In [98]:
# Rotate to asparagus angle
rot_result = rotate_about_center(img, -3.4682295322418213)
cv2.imwrite("result_rot.jpg", rot_result)

True

In [99]:
ret2,th_horizontal_asparagus = cv2.threshold(rot_result,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
plt.imshow(th_horizontal_asparagus)
plt.show()
cv2.imwrite("th_horizontal_asparagus.jpg", th_horizontal_asparagus)

True

In [101]:
th_horizontal_asparagus_to_search_contour = th_horizontal_asparagus.copy()
contours, hierarchy = cv2.findContours(th_horizontal_asparagus_to_search_contour,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [102]:
cnt = contours[0]
for act_cnt in contours:
    if cv2.contourArea(act_cnt) > cv2.contourArea(cnt):
        cnt = act_cnt

In [103]:
x,y,w,h = cv2.boundingRect(cnt)

In [104]:
asaragus_contour = th_horizontal_asparagus[y:y + h, x:x + w]

In [105]:
cv2.imwrite("final_contour.jpg", asaragus_contour)

True

In [106]:
columns = (asaragus_contour != 0).sum(0)
rows    = (asaragus_contour != 0).sum(1)

In [114]:
plt.plot(columns)
plt.show()

In [118]:
plt.hist(rows, bins=100)
plt.title("Gaussian Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()